In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow as tf
import time
import os 

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from src.utils.time_series_procs import split_sequences

In [2]:
def get_times_series_data(df, cut_date, time_steps=3):
    columns = list(df.columns.drop(["R28D"]))
    columns.append("R28D")
    df = df.reindex(columns=columns)
    df["Data/Hora"] = pd.to_datetime(df["Data/Hora"])
    df_train = df[df["Data/Hora"] < cut_date]
    df_test = df[df["Data/Hora"] >= cut_date]
    df_train = df_train.drop(["Data/Hora"], axis=1)
    df_test = df_test.drop(["Data/Hora"], axis=1)
    x_train, y_train = split_sequences(df_train.values, time_steps)
    x_test, y_test = split_sequences(df_test.values, time_steps)
    return x_train, y_train, x_test, y_test

In [3]:
class StandardScaler3DShape:
    def __init__(self):
        self.scaler = StandardScaler()

    def fit_transform(self, X, y=None):
        X_new = self.scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

    def fit(self, X, y=None):
        X_new = X.reshape(-1, X.shape[-1])
        self.scaler.fit(X_new)
        return self

    def transform(self, X, y=None):
        X_new = self.scaler.transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)
        return X_new

In [4]:
def get_baseline_model(n_units=64, learning_rate=0.01):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.LSTM(units=n_units))
    model.add(tf.keras.layers.Dense(1))

    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=opt,
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [5]:
def create_keras_model(
    activation="tanh", dropout_rate=0.0, n_units=64, n_layers=1, learning_rate=1e-3
):
    model = tf.keras.Sequential()

    for i in range(n_layers):
        if i + 1 < n_layers:
            model.add(
                tf.keras.layers.LSTM(
                    units=n_units, activation=activation, return_sequences=True
                )
            )
        else:
            model.add(
                tf.keras.layers.LSTM(
                    units=n_units, activation=activation, return_sequences=False
                )
            )
    if dropout_rate > 0.0:
        model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(1))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="mse",
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="RMSE")],
    )
    return model

In [6]:
def make_repeated_time_series_k_fold(
    x,
    y,
    train_period,
    test_period,
    grid=None,
    repeats=10,
    show_individual_results=True,
    show_final_result=True,
    params=None,
):
    results = []
    max_samples = x.shape[0]

    if grid:
        activation = grid.best_params_["model__activation"]
        dropout_rate = grid.best_params_["model__dropout_rate"]
        n_units = grid.best_params_["model__n_units"]
        n_layers = grid.best_params_["model__n_layers"]
        learning_rate = grid.best_params_["model__learning_rate"]
        batch_size = grid.best_params_["model__batch_size"]
        epochs = grid.best_params_["model__epochs"]
    else:
        activation = params["activation"]
        dropout_rate = params["dropout_rate"]
        n_units = params["n_units"]
        n_layers = params["n_layers"]
        learning_rate = params["learning_rate"]
        batch_size = params["batch_size"]
        epochs = params["epochs"]

    for _ in range(repeats):
        i = 0
        scores = []
        for _ in range(0, max_samples // train_period):
            x_train = x[i : i + train_period]
            y_train = y[i : i + train_period]
            x_test = x[i + train_period : i + train_period + test_period]
            y_test = y[i + train_period : i + train_period + test_period]
            i += train_period
            x_train = scaler.fit_transform(
                x_train.reshape(-1, x_train.shape[-1])
            ).reshape(x_train.shape)
            x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(
                x_test.shape
            )
            model = create_keras_model(
                activation=activation,
                dropout_rate=dropout_rate,
                n_units=n_units,
                n_layers=n_layers,
                learning_rate=learning_rate,
            )
            model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
            rmse = model.evaluate(x_test, y_test, verbose=0)
            scores.append(rmse[1])
        results.append(scores)
        if show_individual_results:
            print("RMSE: %.3f (%.3f)" % (np.mean(scores), np.std(scores)))
    if show_final_result:
        print("\nRMSE: %.3f (%.3f)" % (np.mean(results), np.std(results)))
    return results

In [7]:
def make_timesteps_repeated_time_series_k_fold(
    df,
    train_period,
    test_period,
    params=None,
    grid=None,
    repeats=10,
    timesteps_list=[3],
    show_results=True,
):
    results = {}

    for timesteps in timesteps_list:
        x, y = split_sequences(df.values, timesteps)
        scores = make_repeated_time_series_k_fold(
            x, y, train_period, test_period, grid, repeats, False, False, params
        )
        results[timesteps] = scores
        if show_results:
            print(
                "TIMESTEPS: %d RMSE: %.3f (%.3f)"
                % (timesteps, np.mean(scores), np.std(scores))
            )
    return results

<h3>Disable Tensorflow Warnings</h3>

In [8]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# FFT Transform

This notebook contains that experiments with the data transformed with fast fourier transform.

In [9]:
df_r3d_and_r7d = pd.read_csv(
    "../../../../data/interim/"
    + "r3d_and_r7d-no-resampling-with-fillna-and-fft-vars-only-"
    + "dados-historicos-partner_i-cement-CPIIE40.csv"
)
df_r3d_only = pd.read_csv(
    "../../../../data/interim/"
    + "r3d_only-no-resampling-with-fillna-and-fft-vars-only-"
    + "dados-historicos-partner_i-cement-CPIIE40.csv"
)
df_no_r3d_r7d = pd.read_csv(
    "../../../../data/interim/"
    + "no-r3d-r7d-no-resampling-with-fillna-and-fft-vars-only-"
    + "dados-historicos-partner_i-cement-CPIIE40.csv"
)

In [10]:
seed = 47
scaler = StandardScaler()

# Neural Network Long Short Term Memory - LSTM

# Configs:

1. Train Period: [01-01-2019, 2021-05-02)

2. Test Period:  (2021-05-02, 2021-11-31]

3. TIMESTEPS: [1, 3, 5, 7, 10, 15, 20]

4. Epochs: 100

5. Batch size: 32

6. LSTM Cells: 64

7. LSTM Layers: 1

8. Optimization Algorithm: Adam
    
    8.1. Learning Rate: 1e-2
    
<h3>Repeats: 10</h3>
<h3>K: 10</h3>

# Times Series Repeated KFold Cross validation - different Timesteps values

Here we will make a Times Series KFold Cross validation with all 3 datasets. Here we are using the Blocking Time Series Split method. And with timeseries Split method. We also do it with different number of timesteps


In [11]:
params = {}
params['activation'] = 'tanh'
params['dropout_rate'] = 0.0
params['n_units'] = 64
params['n_layers'] = 1
params['learning_rate'] = 1e-2
params['batch_size'] = 32
params['epochs'] = 100

timesteps_list = [1, 3, 5, 7, 10, 15, 20]

repeats = 10

<h3> Time Series Repeated KFold Cross Validation with vary timesteps - df_no_r3d_r7d</h3>

<b>Dataset: df_no_r3d_r7d:</b> In this dataset the R3D and R7D variables are not considered.

In [12]:
df = df_no_r3d_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora'], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [13]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [14]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 17.583 (5.567)
TIMESTEPS: 3 RMSE: 2.408 (0.785)
TIMESTEPS: 5 RMSE: 2.272 (1.100)
TIMESTEPS: 7 RMSE: 2.745 (4.787)
TIMESTEPS: 10 RMSE: 3.151 (5.263)
TIMESTEPS: 15 RMSE: 2.478 (4.566)
TIMESTEPS: 20 RMSE: 2.201 (0.384)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [15]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [16]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 15.723 (1.210)
TIMESTEPS: 3 RMSE: 2.325 (0.129)
TIMESTEPS: 5 RMSE: 4.344 (5.741)
TIMESTEPS: 7 RMSE: 2.411 (0.172)
TIMESTEPS: 10 RMSE: 2.379 (0.127)
TIMESTEPS: 15 RMSE: 1.157 (0.030)
TIMESTEPS: 20 RMSE: 1.539 (0.010)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [17]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [18]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 14.518 (3.495)
TIMESTEPS: 3 RMSE: 3.419 (1.162)
TIMESTEPS: 5 RMSE: 10.253 (15.862)
TIMESTEPS: 7 RMSE: 4.131 (6.199)
TIMESTEPS: 10 RMSE: 1.693 (0.024)
TIMESTEPS: 15 RMSE: 5.454 (10.769)
TIMESTEPS: 20 RMSE: 1.681 (0.014)


<h3> Time Series Repeated KFold Cross Validation - df_r3d_only</h3>

<b>Dataset: df_r3d_only:</b> In this dataset only R3D variable is considered.

In [19]:
df = df_r3d_only.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora'], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [20]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [21]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 17.257 (5.733)
TIMESTEPS: 3 RMSE: 2.440 (0.740)
TIMESTEPS: 5 RMSE: 2.853 (2.335)
TIMESTEPS: 7 RMSE: 5.335 (8.118)
TIMESTEPS: 10 RMSE: 4.464 (6.892)
TIMESTEPS: 15 RMSE: 1.940 (1.576)
TIMESTEPS: 20 RMSE: 2.168 (0.405)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [22]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [23]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 15.928 (1.154)
TIMESTEPS: 3 RMSE: 2.335 (0.083)
TIMESTEPS: 5 RMSE: 4.235 (5.775)
TIMESTEPS: 7 RMSE: 2.430 (0.108)
TIMESTEPS: 10 RMSE: 5.174 (8.287)
TIMESTEPS: 15 RMSE: 1.174 (0.078)
TIMESTEPS: 20 RMSE: 1.565 (0.074)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [24]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [25]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 14.212 (2.266)
TIMESTEPS: 3 RMSE: 4.793 (2.830)
TIMESTEPS: 5 RMSE: 12.864 (17.767)
TIMESTEPS: 7 RMSE: 9.783 (12.034)
TIMESTEPS: 10 RMSE: 1.714 (0.114)
TIMESTEPS: 15 RMSE: 1.784 (0.012)
TIMESTEPS: 20 RMSE: 1.673 (0.012)


<h3> Time Series Repeated KFold Cross Validation - df_r3d_and_r7d</h3>

<b>Dataset: df_r3d_and_r7d:</b> In this dataset both R3D and R7D variables are considered.

In [26]:
df = df_r3d_and_r7d.copy()
columns = list(df.columns.drop(['R28D']))
columns.append('R28D')
df = df.reindex(columns=columns)
df = df.drop(['Data/Hora'], axis=1)

<h4>Experiment 1</h4>
    
<b>Train 6 months and predicts one month every year</b>

In [27]:
train_period = 90 # 6 months of data since the data is spaced every 2 days
test_period = 15 # 1 month

In [28]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 17.458 (5.509)
TIMESTEPS: 3 RMSE: 2.767 (1.104)
TIMESTEPS: 5 RMSE: 2.447 (1.786)
TIMESTEPS: 7 RMSE: 3.775 (7.274)
TIMESTEPS: 10 RMSE: 4.452 (7.800)
TIMESTEPS: 15 RMSE: 1.718 (0.591)
TIMESTEPS: 20 RMSE: 2.151 (0.417)


<h4>Experiment 2</h4>
    
<b>Train 11 months and predicts one month every year</b>

In [29]:
train_period = 165 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 15 # 1 month

In [30]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 15.373 (1.033)
TIMESTEPS: 3 RMSE: 2.277 (0.099)
TIMESTEPS: 5 RMSE: 2.327 (0.129)
TIMESTEPS: 7 RMSE: 4.670 (7.013)
TIMESTEPS: 10 RMSE: 2.351 (0.019)
TIMESTEPS: 15 RMSE: 1.150 (0.017)
TIMESTEPS: 20 RMSE: 1.605 (0.164)


<h4>Experiment 3</h4>
    
<b>Train 2 years and 10 months and predicts october and november of 2021</b>

In [31]:
train_period = 200 # 11 months of data since the data is spaced every 2 days (180 + 150)/2
test_period = 30 # 2 months

In [32]:
results = make_timesteps_repeated_time_series_k_fold(
    df, train_period, test_period, params, None, repeats, timesteps_list
)

TIMESTEPS: 1 RMSE: 13.280 (1.828)
TIMESTEPS: 3 RMSE: 4.814 (3.305)
TIMESTEPS: 5 RMSE: 7.716 (9.067)
TIMESTEPS: 7 RMSE: 4.413 (5.923)
TIMESTEPS: 10 RMSE: 5.087 (6.920)
TIMESTEPS: 15 RMSE: 1.777 (0.011)
TIMESTEPS: 20 RMSE: 1.684 (0.018)
